In [1]:
from evaluate import *
from generate_data import *
import time
from focal_Loss import *

/home/leonaire/anaconda3/envs/ecole/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
        
scip_parameters = {"branching/scorefunc": "s",
                   "separating/maxrounds": 0,
                   "limits/time": 360,
                   "conflict/enable": False,
                   "presolving/maxrounds": 0,
                   "presolving/maxrestarts": 0,
                   "separating/maxroundsroot" : 0,
                   "separating/maxcutsroot": 0,
                   "separating/maxcuts": 0,
                   "propagating/maxroundsroot" : 0,
                   "lp/presolving" : False,
                  }

# generate_dataset(scip_parameters,path = "DataSetEvaluate/",nb_cons = [800,1000],nb_var = [800,1000],density = [0.1,0.2],nb_instance=20,only_problem=True)

In [3]:
def is_realisable(solu,A):
    size_var = len(solu)
    size_cons = A.shape[0] - size_var
    Aux = A[:size_var,size_cons:]
    for i,v in enumerate(solu):
        if v:
            for i_cons in np.where(Aux[i]!=0)[0][::-1]:
                Aux = np.delete(Aux, i_cons, axis=1)
    return Aux.shape[1]

In [4]:
def fix_and_optimize(var_fix_one,var_fix_zero,lp_file):
    solver = ecole.scip.Model.from_file(lp_file)
    aspyscip = solver.as_pyscipopt()
    aspyscip.setPresolve(pyscip.SCIP_PARAMSETTING.OFF)
    for i in var_fix_one:
        aspyscip.fixVar(aspyscip.getVars()[i],1)
    for i in var_fix_zero:
        aspyscip.fixVar(aspyscip.getVars()[i],0)
    aspyscip.optimize()
    return aspyscip

def gnn_plus_scip(scip_parameters,model,lp_file,threshold_one=0.9,threshold_zero=10e-5):
    H,A,nb_var = getInput(scip_parameters,lp)
    y_hat,predictions = evaluate(model,H,A,nb_var,scip_parameters)
    predictions = predictions.detach().numpy()
    var_fixe_one,var_fixe_zero = [],[]
    for i,v in enumerate(predictions):
        if v < threshold_zero:
            var_fixe_zero.append(i)
        elif v > threshold_one:
            var_fixe_one.append(i)
    print("fix %d one and %d zero"%(len(var_fixe_one),len(var_fixe_zero)))
    return fix_and_optimize(var_fixe_one,var_fixe_zero,lp_file)


In [33]:
path_set = os.listdir("DataSetEvaluate/")
dataSet = []
hist = []#list of vector [is_gnn_realisable,gap_gnn,gap_heuristique,gap_scip,temp_gnn,temp_scip]
idx=0
for filename in path_set:
    lp = "DataSetEvaluate/"+filename
    train_name = "SC1000_BCE_3MLP_50TailleH_1batch_size_layernorm+cat"
    aux = []
    
    time_start=time.time()
    model_path = "model/"+train_name
    criterion = nn.BCELoss()
    net = VariablePredictor(24,50,3)
    net.load_state_dict(torch.load(model_path,map_location=torch.device('cpu')))
    res_her,res_gnn,A = optimize(scip_parameters,net,lp)
    time_gnn = time.time() - time_start
    
    time_start=time.time()
    aspyscip_plus = gnn_plus_scip(scip_parameters,net,lp)
    res = \
        np.array(np.around(list(ast.literal_eval(aspyscip_plus.getBestSol().__repr__()).values()), 0).astype(int).tolist())
    time_plus = time.time() - time_start
    gap_plus = aspyscip_plus.getGap()
    
    
    aux.append(is_realisable(res_gnn,A))
    if aux[0] != 0:
        aux.append(1)
    else:
        for i,v in enumerate(res_gnn):
            aspyscip_plus.setSolVal(aspyscip_plus.getBestSol(),aspyscip_plus.getVars()[i],v)
        aux.append(abs(aspyscip_plus.getDualbound() - aspyscip_plus.getObjVal())/min(aspyscip_plus.getDualbound(),aspyscip_plus.getObjVal()))
    
    for i,v in enumerate(res_her):
        aspyscip_plus.setSolVal(aspyscip.getBestSol(),aspyscip_plus.getVars()[i],v)
    aux.append(abs(aspyscip_plus.getDualbound() - aspyscip_plus.getObjVal())/min(aspyscip_plus.getDualbound(),aspyscip_plus.getObjVal()))
    
    time_start=time.time()
    solver = ecole.scip.Model.from_file(lp)
    aspyscip = solver.as_pyscipopt()
    aspyscip.setPresolve(pyscip.SCIP_PARAMSETTING.OFF)
    aspyscip.optimize()
    res_scip = np.array(np.around(list(ast.literal_eval(aspyscip.getBestSol().__repr__()).values()), 0).astype(int).tolist())
    time_scip = time.time() - time_start
    gap_scip = aspyscip.getGap()
    
    aux.append(gap_plus)    
    aux.append(gap_scip)
    aux.append(time_gnn)    
    aux.append(time_plus)
    aux.append(time_scip)
    hist.append(aux)
    print("Instance %d|ConsResteGnn=%f,Gap_GNN=%f,Gap_Her=%f,Gap_Plus=%f,gap_SCIP=%f \n temp_GNN=%f,temp_GNN=%f,temp_SCIP=%f"\
              %(idx,aux[0],aux[1],aux[2],aux[3],aux[4],aux[5],aux[6],aux[7]))
    idx+=1
np.mean(hist, axis=0)
hist_SC = np.rot90(hist, -1)


fix 16 one and 721 zero
Instance 0|ConsResteGnn=0.000000,Gap_GNN=0.000000,Gap_Her=0.000000,Gap_Plus=0.000000,gap_SCIP=0.000000 
 temp_GNN=3.677859,temp_GNN=2.832854,temp_SCIP=15.469497
fix 10 one and 941 zero
Instance 1|ConsResteGnn=195.000000,Gap_GNN=1.000000,Gap_Her=0.000000,Gap_Plus=0.000000,gap_SCIP=0.000000 
 temp_GNN=5.654935,temp_GNN=3.402207,temp_SCIP=10.524505
fix 18 one and 651 zero
Instance 2|ConsResteGnn=0.000000,Gap_GNN=0.000000,Gap_Her=0.000000,Gap_Plus=0.000000,gap_SCIP=0.000000 
 temp_GNN=3.941860,temp_GNN=4.412230,temp_SCIP=25.347491
fix 16 one and 834 zero
Instance 3|ConsResteGnn=0.000000,Gap_GNN=0.000000,Gap_Her=0.000000,Gap_Plus=0.000000,gap_SCIP=0.000000 
 temp_GNN=4.076076,temp_GNN=5.392914,temp_SCIP=45.792162
fix 15 one and 735 zero
Instance 4|ConsResteGnn=0.000000,Gap_GNN=0.000000,Gap_Her=0.000000,Gap_Plus=0.000000,gap_SCIP=0.000000 
 temp_GNN=2.630487,temp_GNN=2.697573,temp_SCIP=12.334457
fix 17 one and 944 zero
Instance 5|ConsResteGnn=0.000000,Gap_GNN=0.000000

Instance 44|ConsResteGnn=193.000000,Gap_GNN=1.000000,Gap_Her=0.000000,Gap_Plus=0.000000,gap_SCIP=0.000000 
 temp_GNN=5.462926,temp_GNN=2.579878,temp_SCIP=5.586052
fix 12 one and 673 zero
Instance 45|ConsResteGnn=0.000000,Gap_GNN=0.000000,Gap_Her=0.000000,Gap_Plus=0.000000,gap_SCIP=0.000000 
 temp_GNN=4.160746,temp_GNN=6.407442,temp_SCIP=74.665022
fix 23 one and 653 zero
Instance 46|ConsResteGnn=0.000000,Gap_GNN=0.000000,Gap_Her=0.000000,Gap_Plus=0.000000,gap_SCIP=0.000000 
 temp_GNN=3.289101,temp_GNN=2.693450,temp_SCIP=43.547880
fix 17 one and 892 zero
Instance 47|ConsResteGnn=195.000000,Gap_GNN=1.000000,Gap_Her=0.000000,Gap_Plus=0.000000,gap_SCIP=0.000000 
 temp_GNN=4.530037,temp_GNN=3.323782,temp_SCIP=17.010875
fix 26 one and 657 zero
Instance 48|ConsResteGnn=0.000000,Gap_GNN=0.000000,Gap_Her=0.000000,Gap_Plus=0.000000,gap_SCIP=0.000000 
 temp_GNN=3.155875,temp_GNN=1.886746,temp_SCIP=9.085449
fix 17 one and 617 zero
Instance 49|ConsResteGnn=0.000000,Gap_GNN=0.015152,Gap_Her=0.015152,

Instance 88|ConsResteGnn=0.000000,Gap_GNN=0.000000,Gap_Her=0.000000,Gap_Plus=0.000000,gap_SCIP=0.000000 
 temp_GNN=2.969009,temp_GNN=2.275568,temp_SCIP=8.909396
fix 21 one and 889 zero
Instance 89|ConsResteGnn=187.000000,Gap_GNN=1.000000,Gap_Her=0.000000,Gap_Plus=0.000000,gap_SCIP=0.000000 
 temp_GNN=5.257703,temp_GNN=2.054810,temp_SCIP=9.027630
fix 16 one and 698 zero
Instance 90|ConsResteGnn=0.000000,Gap_GNN=0.000000,Gap_Her=0.000000,Gap_Plus=0.000000,gap_SCIP=0.000000 
 temp_GNN=3.889216,temp_GNN=3.969163,temp_SCIP=44.335430
fix 19 one and 849 zero
Instance 91|ConsResteGnn=0.000000,Gap_GNN=0.000000,Gap_Her=0.000000,Gap_Plus=0.000000,gap_SCIP=0.000000 
 temp_GNN=4.244133,temp_GNN=3.334806,temp_SCIP=13.570946
fix 11 one and 881 zero
Instance 92|ConsResteGnn=195.000000,Gap_GNN=1.000000,Gap_Her=0.000000,Gap_Plus=0.000000,gap_SCIP=0.000000 
 temp_GNN=6.850028,temp_GNN=6.219397,temp_SCIP=89.562760
fix 16 one and 759 zero
Instance 93|ConsResteGnn=0.000000,Gap_GNN=0.000000,Gap_Her=0.000000,

Instance 132|ConsResteGnn=0.000000,Gap_GNN=0.000000,Gap_Her=0.000000,Gap_Plus=0.000000,gap_SCIP=0.000000 
 temp_GNN=3.528337,temp_GNN=2.787014,temp_SCIP=30.371471
fix 17 one and 820 zero
Instance 133|ConsResteGnn=0.000000,Gap_GNN=0.008772,Gap_Her=0.008772,Gap_Plus=0.000000,gap_SCIP=0.000000 
 temp_GNN=6.284723,temp_GNN=7.207066,temp_SCIP=54.351565
fix 20 one and 612 zero
Instance 134|ConsResteGnn=0.000000,Gap_GNN=0.014388,Gap_Her=0.014388,Gap_Plus=0.000000,gap_SCIP=0.000000 
 temp_GNN=5.992256,temp_GNN=8.232137,temp_SCIP=104.391343
fix 24 one and 617 zero
Instance 135|ConsResteGnn=0.000000,Gap_GNN=0.007752,Gap_Her=0.007752,Gap_Plus=0.000000,gap_SCIP=0.000000 
 temp_GNN=4.025243,temp_GNN=3.550889,temp_SCIP=125.660044
fix 23 one and 637 zero
Instance 136|ConsResteGnn=0.000000,Gap_GNN=0.000000,Gap_Her=0.000000,Gap_Plus=0.000000,gap_SCIP=0.000000 
 temp_GNN=4.788046,temp_GNN=3.277635,temp_SCIP=19.546960
fix 20 one and 710 zero
Instance 137|ConsResteGnn=0.000000,Gap_GNN=0.019417,Gap_Her=0.0

In [39]:
print("Pourcentage de solutions realisables de GNN :",len(np.where(hist_SC[0] == 0)[0])/len(hist_SC[0]))
print("Gap des solutions realisables de GNN :",(sum(hist_SC[1])-len(np.where(hist_SC[1] == 1.0)[0]))/len(np.where(hist_SC[1] != 1.0)[0]))
print("Pourcentage de solutions bonnes de GNN+Heuristique",len(np.where(hist_SC[2]  < 1)[0])/len(hist_SC[2]))
print("Gap des solutions de GNN+Heuristique:",(sum([i if i < 1 else 0 for i in hist_SC[2]])/len(np.where(hist_SC[2]  < 1)[0])))
print("Temps pour GNN:%f"%np.mean(hist_SC[5]))
print("Temps pour GNN_SCIP:%f"%np.mean(hist_SC[6]))
print("Temps pour SICP:%f"%np.mean(hist_SC[7]))

Pourcentage de solutions realisables de GNN : 0.75
Gap des solutions realisables de GNN : 0.0037557552473475773
Pourcentage de solutions bonnes de GNN+Heuristique 1.0
Gap des solutions de GNN+Heuristique: 0.002816816435510653
Temps pour GNN:4.345574
Temps pour GNN_SCIP:3.849391
Temps pour SICP:26.866530


In [40]:
path_set = os.listdir("DataSet_CFL/")
dataSet = []
hist = []#list of vector [is_gnn_realisable,gap_gnn,gap_heuristique,gap_scip,temp_gnn,temp_scip]
idx=0
for filename in path_set:
    if idx<100:
        lp = "DataSet_CFL/"+filename+"/problem.lp"
        train_name = "CFL30_FL_0.5Gamma_0.25Alpha_6MLP_50TailleH_1batch_size_1-10e-6_schedulerlayernorm+cat"
        model_path = "model/"+train_name
        net = VariablePredictor(24,50,6)
        net.load_state_dict(torch.load(model_path,map_location=torch.device('cpu')))
        aux = []

        time_start=time.time()
        aspyscip_plus = gnn_plus_scip(scip_parameters,net,lp,0.9,0)
        if (aspyscip_plus.getStatus() == "optimal"):
            res = \
                np.array(np.around(list(ast.literal_eval(aspyscip_plus.getBestSol().__repr__()).values()), 0).astype(int).tolist())
            time_plus = time.time() - time_start
            gap_plus = aspyscip_plus.getGap()
        else:
            time_plus = -1
            gap_plus = 1

        time_start=time.time()
        solver = ecole.scip.Model.from_file(lp)
        aspyscip = solver.as_pyscipopt()
        aspyscip.setPresolve(pyscip.SCIP_PARAMSETTING.OFF)
        aspyscip.optimize()
        res_scip = np.array(np.around(list(ast.literal_eval(aspyscip.getBestSol().__repr__()).values()), 0).astype(int).tolist())
        time_scip = time.time() - time_start
        gap_scip = aspyscip.getGap()

        aux.append(gap_plus)    
        aux.append(gap_scip) 
        aux.append(time_plus)
        aux.append(time_scip)
        hist.append(aux)
        print("Instance %d|Gap_Plus=%f,gap_SCIP=%f \n temp_GNN=%f,temp_SCIP=%f"\
                  %(idx,aux[0],aux[1],aux[2],aux[3]))
    idx+=1
np.mean(hist, axis=0)
hist_CFL = np.rot90(hist, -1)


fix 26 one and 0 zero
Instance 0|Gap_Plus=0.000000,gap_SCIP=0.000000 
 temp_GNN=0.631356,temp_SCIP=12.324170
fix 28 one and 0 zero
Instance 1|Gap_Plus=1.000000,gap_SCIP=0.000000 
 temp_GNN=-1.000000,temp_SCIP=16.740614
fix 16 one and 0 zero
Instance 2|Gap_Plus=0.000000,gap_SCIP=0.000000 
 temp_GNN=1.255213,temp_SCIP=24.024290
fix 22 one and 0 zero
Instance 3|Gap_Plus=0.000000,gap_SCIP=0.000000 
 temp_GNN=0.296172,temp_SCIP=5.324076
fix 25 one and 0 zero
Instance 4|Gap_Plus=0.000000,gap_SCIP=0.000000 
 temp_GNN=0.286872,temp_SCIP=16.820111
fix 30 one and 0 zero
Instance 5|Gap_Plus=0.000000,gap_SCIP=0.000000 
 temp_GNN=0.551279,temp_SCIP=2.148664
fix 22 one and 0 zero
Instance 6|Gap_Plus=0.000000,gap_SCIP=0.000000 
 temp_GNN=2.768505,temp_SCIP=22.707666
fix 27 one and 0 zero
Instance 7|Gap_Plus=0.000000,gap_SCIP=0.000000 
 temp_GNN=0.958927,temp_SCIP=22.399829
fix 27 one and 0 zero
Instance 8|Gap_Plus=0.000000,gap_SCIP=0.000000 
 temp_GNN=0.315382,temp_SCIP=14.311342
fix 35 one and 0 zer

fix 30 one and 0 zero
Instance 75|Gap_Plus=0.000000,gap_SCIP=0.000000 
 temp_GNN=0.634134,temp_SCIP=13.240960
fix 22 one and 0 zero
Instance 76|Gap_Plus=0.000000,gap_SCIP=0.000000 
 temp_GNN=0.303249,temp_SCIP=1.901953
fix 29 one and 0 zero
Instance 77|Gap_Plus=0.000000,gap_SCIP=0.000000 
 temp_GNN=0.430258,temp_SCIP=14.216782
fix 29 one and 0 zero
Instance 78|Gap_Plus=0.000000,gap_SCIP=0.000000 
 temp_GNN=0.442267,temp_SCIP=12.904282
fix 23 one and 0 zero
Instance 79|Gap_Plus=0.000000,gap_SCIP=0.000000 
 temp_GNN=0.367555,temp_SCIP=2.126403
fix 22 one and 0 zero
Instance 80|Gap_Plus=0.000000,gap_SCIP=0.000000 
 temp_GNN=0.303564,temp_SCIP=8.221297
fix 27 one and 0 zero
Instance 81|Gap_Plus=0.000000,gap_SCIP=0.000000 
 temp_GNN=0.646141,temp_SCIP=15.366991
fix 24 one and 0 zero
Instance 82|Gap_Plus=0.000000,gap_SCIP=0.000000 
 temp_GNN=0.233093,temp_SCIP=1.188228
fix 30 one and 0 zero
Instance 83|Gap_Plus=0.000000,gap_SCIP=0.000000 
 temp_GNN=0.290975,temp_SCIP=2.886241
fix 25 one and 

In [53]:
print("Pourcentage de solutions realisables de GNN + Scip:",len(np.where(hist_CFL[0] != 1.00)[0])/len(hist_CFL[0]))
print("Gap des solutions realisables de GNN + Scip :",(sum(hist_CFL[0])-len(np.where(hist_CFL[0] == 1.0)[0]))/len(np.where(hist_CFL[0] != 1.0)[0]))
print("Temps pour GNN + Scip:%f"%((sum(hist_CFL[2])+len(np.where(hist_CFL[0] == -1.0)[0]))/len(np.where(hist_CFL[0] != 1.0)[0])))
print("Temps pour SCIP:%f"%np.mean(hist_CFL[3]))

Pourcentage de solutions realisables de GNN + Scip: 0.74
Gap des solutions realisables de GNN + Scip : 0.0
Temps pour GNN + Scip:0.953414
Temps pour SCIP:13.879614


In [46]:
path_set = os.listdir("DataSet_CFL/")
dataSet = []
hist = []#list of vector [is_gnn_realisable,gap_gnn,gap_heuristique,gap_scip,temp_gnn,temp_scip]
idx=0
for filename in path_set:
    if idx<100:
        lp = "DataSet_CFL/"+filename+"/problem.lp"
        train_name = "CFL30_BCE_6MLP_50TailleH_1batch_size_layernorm+cat+scheduler"
        model_path = "model/"+train_name
        net = VariablePredictor(24,50,6)
        net.load_state_dict(torch.load(model_path,map_location=torch.device('cpu')))
        aux = []

        time_start=time.time()
        aspyscip_plus = gnn_plus_scip(scip_parameters,net,lp,0.9,0)
        if (aspyscip_plus.getStatus() == "optimal"):
            res = \
                np.array(np.around(list(ast.literal_eval(aspyscip_plus.getBestSol().__repr__()).values()), 0).astype(int).tolist())
            time_plus = time.time() - time_start
            gap_plus = aspyscip_plus.getGap()
        else:
            time_plus = -1
            gap_plus = 1

        time_start=time.time()
        solver = ecole.scip.Model.from_file(lp)
        aspyscip = solver.as_pyscipopt()
        aspyscip.setPresolve(pyscip.SCIP_PARAMSETTING.OFF)
        aspyscip.optimize()
        res_scip = np.array(np.around(list(ast.literal_eval(aspyscip.getBestSol().__repr__()).values()), 0).astype(int).tolist())
        time_scip = time.time() - time_start
        gap_scip = aspyscip.getGap()

        aux.append(gap_plus)    
        aux.append(gap_scip) 
        aux.append(time_plus)
        aux.append(time_scip)
        hist.append(aux)
        print("Instance %d|Gap_Plus=%f,gap_SCIP=%f \n temp_GNN=%f,temp_SCIP=%f"\
                  %(idx,aux[0],aux[1],aux[2],aux[3]))
    idx+=1
np.mean(hist, axis=0)
hist_CFL = np.rot90(hist, -1)

fix 22 one and 0 zero
Instance 0|Gap_Plus=0.000000,gap_SCIP=0.000000 
 temp_GNN=0.586061,temp_SCIP=13.245099
fix 13 one and 0 zero
Instance 1|Gap_Plus=0.000000,gap_SCIP=0.000000 
 temp_GNN=0.643669,temp_SCIP=16.970366
fix 23 one and 0 zero
Instance 2|Gap_Plus=0.000000,gap_SCIP=0.000000 
 temp_GNN=0.438555,temp_SCIP=24.170418
fix 25 one and 0 zero
Instance 3|Gap_Plus=0.000000,gap_SCIP=0.000000 
 temp_GNN=0.230624,temp_SCIP=5.225989
fix 16 one and 0 zero
Instance 4|Gap_Plus=1.000000,gap_SCIP=0.000000 
 temp_GNN=-1.000000,temp_SCIP=16.858236
fix 17 one and 0 zero
Instance 5|Gap_Plus=0.000000,gap_SCIP=0.000000 
 temp_GNN=0.511254,temp_SCIP=1.939230
fix 23 one and 0 zero
Instance 6|Gap_Plus=1.000000,gap_SCIP=0.000000 
 temp_GNN=-1.000000,temp_SCIP=23.080796
fix 12 one and 0 zero
Instance 7|Gap_Plus=0.000000,gap_SCIP=0.000000 
 temp_GNN=1.659896,temp_SCIP=22.275145
fix 13 one and 0 zero
Instance 8|Gap_Plus=0.000000,gap_SCIP=0.000000 
 temp_GNN=0.342090,temp_SCIP=9.636956
fix 9 one and 0 zero

fix 10 one and 0 zero
Instance 75|Gap_Plus=0.000000,gap_SCIP=0.000000 
 temp_GNN=4.228601,temp_SCIP=12.562457
fix 35 one and 0 zero
Instance 76|Gap_Plus=0.000000,gap_SCIP=0.000000 
 temp_GNN=0.203294,temp_SCIP=1.831798
fix 14 one and 0 zero
Instance 77|Gap_Plus=0.000000,gap_SCIP=0.000000 
 temp_GNN=0.531472,temp_SCIP=13.794885
fix 18 one and 0 zero
Instance 78|Gap_Plus=0.000000,gap_SCIP=0.000000 
 temp_GNN=0.459658,temp_SCIP=12.789782
fix 11 one and 0 zero
Instance 79|Gap_Plus=0.000000,gap_SCIP=0.000000 
 temp_GNN=0.205026,temp_SCIP=2.177919
fix 21 one and 0 zero
Instance 80|Gap_Plus=0.000000,gap_SCIP=0.000000 
 temp_GNN=0.321290,temp_SCIP=8.034435
fix 19 one and 0 zero
Instance 81|Gap_Plus=0.000000,gap_SCIP=0.000000 
 temp_GNN=3.984394,temp_SCIP=15.078002
fix 14 one and 0 zero
Instance 82|Gap_Plus=0.000000,gap_SCIP=0.000000 
 temp_GNN=0.275153,temp_SCIP=1.095963
fix 25 one and 0 zero
Instance 83|Gap_Plus=1.000000,gap_SCIP=0.000000 
 temp_GNN=-1.000000,temp_SCIP=2.937905
fix 30 one and

In [51]:
print("Pourcentage de solutions realisables de GNN + Scip:",len(np.where(hist_CFL[0] != 1.00)[0])/len(hist_CFL[0]))
print("Gap des solutions realisables de GNN + Scip :",(sum(hist_CFL[0])-len(np.where(hist_CFL[0] == 1.0)[0]))/len(np.where(hist_CFL[0] != 1.0)[0]))
print("Temps pour GNN + Scip:%f"%((sum(hist_CFL[2])+len(np.where(hist_CFL[0] == -1.0)[0]))/len(np.where(hist_CFL[0] != 1.0)[0])))
print("Temps pour SCIP:%f"%np.mean(hist_CFL[3]))

Pourcentage de solutions realisables de GNN + Scip: 0.87
Gap des solutions realisables de GNN + Scip : 0.0
Temps pour GNN + Scip:0.940878
Temps pour SCIP:13.909654


In [6]:
path_set = os.listdir("DataSet3/")
dataSet = []
hist = []#list of vector [is_gnn_realisable,gap_gnn,gap_heuristique,gap_scip,temp_gnn,temp_scip]
idx=0
for filename in path_set:
    if idx<100:
        lp = "DataSet3/"+filename+"/problem.lp"
        train_name = "SC1000_BCE_3MLP_50TailleH_1batch_size_layernorm+cat"
        aux = []
        
        time_start=time.time()
        model_path = "model/"+train_name
        criterion = nn.BCELoss()
        net = VariablePredictor(24,50,3)
        net.load_state_dict(torch.load(model_path,map_location=torch.device('cpu')))
        res_her,res_gnn,A = optimize(scip_parameters,net,lp)
        time_gnn = time.time() - time_start

        time_start=time.time()
        aspyscip_plus = gnn_plus_scip(scip_parameters,net,lp)
        res = \
            np.array(np.around(list(ast.literal_eval(aspyscip_plus.getBestSol().__repr__()).values()), 0).astype(int).tolist())
        time_plus = time.time() - time_start
        gap_plus = aspyscip_plus.getGap()


        aux.append(is_realisable(res_gnn,A))
        if aux[0] != 0:
            aux.append(1)
        else:
            for i,v in enumerate(res_gnn):
                aspyscip_plus.setSolVal(aspyscip_plus.getBestSol(),aspyscip_plus.getVars()[i],v)
            aux.append(abs(aspyscip_plus.getDualbound() - aspyscip_plus.getObjVal())/min(aspyscip_plus.getDualbound(),aspyscip_plus.getObjVal()))

        for i,v in enumerate(res_her):
            aspyscip_plus.setSolVal(aspyscip.getBestSol(),aspyscip_plus.getVars()[i],v)
        aux.append(abs(aspyscip_plus.getDualbound() - aspyscip_plus.getObjVal())/min(aspyscip_plus.getDualbound(),aspyscip_plus.getObjVal()))

        time_start=time.time()
        solver = ecole.scip.Model.from_file(lp)
        aspyscip = solver.as_pyscipopt()
        aspyscip.setPresolve(pyscip.SCIP_PARAMSETTING.OFF)
        aspyscip.optimize()
        res_scip = np.array(np.around(list(ast.literal_eval(aspyscip.getBestSol().__repr__()).values()), 0).astype(int).tolist())
        time_scip = time.time() - time_start
        gap_scip = aspyscip.getGap()

        aux.append(gap_plus)    
        aux.append(gap_scip)
        aux.append(time_gnn)    
        aux.append(time_plus)
        aux.append(time_scip)
        hist.append(aux)
        print("Instance %d|ConsResteGnn=%f,Gap_GNN=%f,Gap_Her=%f,Gap_Plus=%f,gap_SCIP=%f \n temp_GNN=%f,temp_GNN=%f,temp_SCIP=%f"\
                  %(idx,aux[0],aux[1],aux[2],aux[3],aux[4],aux[5],aux[6],aux[7]))
        idx+=1
    np.mean(hist, axis=0)
    hist_SC = np.rot90(hist, -1)

pressed CTRL-C 1 times (5 times for forcing termination)
fix 5 one and 1418 zero



KeyboardInterrupt



In [9]:
path_set = os.listdir("DataSet3/")
dataSet = []
hist = []#list of vector [is_gnn_realisable,gap_gnn,gap_heuristique,gap_scip,temp_gnn,temp_scip]
idx=0
for filename in path_set:
    if idx<100:
        lp = "DataSet3/"+filename+"/problem.lp"
        train_name = "SC1000_BCE_3MLP_50TailleH_1batch_size_layernorm+cat"
        aux = []
        
        time_start=time.time()
        model_path = "model/"+train_name
        criterion = nn.BCELoss()
        net = VariablePredictor(24,50,3)
        net.load_state_dict(torch.load(model_path,map_location=torch.device('cpu')))
        res_her,res_gnn,A = optimize(scip_parameters,net,lp)
        time_gnn = time.time() - time_start
        print(time_gnn)
        time_start=time.time()
        aspyscip_plus = gnn_plus_scip(scip_parameters,net,lp)
        res = \
            np.array(np.around(list(ast.literal_eval(aspyscip_plus.getBestSol().__repr__()).values()), 0).astype(int).tolist())
        time_plus = time.time() - time_start
        gap_plus = aspyscip_plus.getGap()
        print(time_plus)

        aux.append(is_realisable(res_gnn,A))
        if aux[0] != 0:
            aux.append(1)
        else:
            for i,v in enumerate(res_gnn):
                aspyscip_plus.setSolVal(aspyscip_plus.getBestSol(),aspyscip_plus.getVars()[i],v)
            aux.append(abs(aspyscip_plus.getDualbound() - aspyscip_plus.getObjVal())/min(aspyscip_plus.getDualbound(),aspyscip_plus.getObjVal()))

        for i,v in enumerate(res_her):
            aspyscip_plus.setSolVal(aspyscip_plus.getBestSol(),aspyscip_plus.getVars()[i],v)
        aux.append(abs(aspyscip_plus.getDualbound() - aspyscip_plus.getObjVal())/min(aspyscip_plus.getDualbound(),aspyscip_plus.getObjVal()))

        time_start=time.time()
        solver = ecole.scip.Model.from_file(lp)
        aspyscip = solver.as_pyscipopt()
        aspyscip.setPresolve(pyscip.SCIP_PARAMSETTING.OFF)
        aspyscip.optimize()
        res_scip = np.array(np.around(list(ast.literal_eval(aspyscip.getBestSol().__repr__()).values()), 0).astype(int).tolist())
        time_scip = time.time() - time_start
        gap_scip = aspyscip.getGap()
        print(time_scip)
    
        aux.append(gap_plus)    
        aux.append(gap_scip)
        aux.append(time_gnn)    
        aux.append(time_plus)
        aux.append(time_scip)
        hist.append(aux)
        print("Instance %d|ConsResteGnn=%f,Gap_GNN=%f,Gap_Her=%f,Gap_Plus=%f,gap_SCIP=%f \n temp_GNN=%f,temp_GNN=%f,temp_SCIP=%f"\
                  %(idx,aux[0],aux[1],aux[2],aux[3],aux[4],aux[5],aux[6],aux[7]))
        idx+=1
    np.mean(hist, axis=0)
    hist_SC = np.rot90(hist, -1)

20.87960410118103
fix 5 one and 1412 zero
428.4683108329773
1623.569135427475
Instance 0|ConsResteGnn=534.000000,Gap_GNN=1.000000,Gap_Her=307.132530,Gap_Plus=0.000000,gap_SCIP=0.000000 
 temp_GNN=20.879604,temp_GNN=428.468311,temp_SCIP=1623.569135
21.130077362060547
fix 10 one and 1389 zero
17.800804138183594
175.20109605789185
Instance 1|ConsResteGnn=510.000000,Gap_GNN=1.000000,Gap_Her=435.305085,Gap_Plus=0.000000,gap_SCIP=0.000000 
 temp_GNN=21.130077,temp_GNN=17.800804,temp_SCIP=175.201096
9.17195725440979
fix 8 one and 1046 zero
36.469929456710815
286.11970114707947
Instance 2|ConsResteGnn=94.000000,Gap_GNN=1.000000,Gap_Her=0.031746,Gap_Plus=0.000000,gap_SCIP=0.000000 
 temp_GNN=9.171957,temp_GNN=36.469929,temp_SCIP=286.119701
24.601670265197754
fix 5 one and 1690 zero
28.54756259918213
244.86195945739746
Instance 3|ConsResteGnn=497.000000,Gap_GNN=1.000000,Gap_Her=894.821429,Gap_Plus=0.000000,gap_SCIP=0.000000 
 temp_GNN=24.601670,temp_GNN=28.547563,temp_SCIP=244.861959
22.35547208


KeyboardInterrupt

